# ALEX LGBl csv export to BibTeX

This converts a CSV file to a bibtex file and was made for mrp-bib Zotero library (https://www.zotero.org/groups/2042149/mrp-bib/library). 


Assumption - CSV file has the following column names:                    

```
SELECT 
	id,			# lfnr, decimal(23,0)
	aid,		# kürzel, varchar(3)
	jahr,		# jahr, varchar(4)
	ausdatvf,	# ausgabedatum als text, varchar(255)
	ausdatsf,	# ausgabedatum als yyyymmdd, varchar(8)
	gesdatvf,	# gesetzesdatum als text, varchar(255)
	gesdatsf,	# gesetzesdatum als yyyymmdd, varchar(8)
	wirdatvf,	# wirkugsdatum als text, varchar(255)
	wirdatsf,	# wirkugsdatum als yyyymmdd, varchar(8)
	gestyp,		# typ [verordnung, gesetz,....], varchar(100)
	inhalt,		# Gesetz, longtext(65535)
	stueck,		# Stück, varchar(50)
	stuecknr,	# Stück als Zahl decimal(22,0),  
	gesnr,		# GesetzesNr, decimal(22,0)
	seite,		# SeitenNr, decimal(22,0)		==> URL:seite=%08d, seite. 
	geschzahl,  # ?, varchar(100)
	abtnr,		# ?, decimal(22,0)
	inst,		# ?, longtext(65535)
	ordner,		# der physikalische unterordner ==> URL:datum=%d%d, jahr, substr(ordner,9,8)
	sprache,	# Sprache [deu, ita, ...], char(3)
	schrift,	# Schrift [L=Latein, K=Kyrillisch], char(1)
	gid,		# Gültigkeit [B, H, R], char(1)
	bezeichnung,# Bezeichnung, varchar(100)
	kurzform, 	# Kurzbezeichnung, varchar(100)
	bool_exists	# es gibt gesetze, da haben wir nicht die entsprechenden seiten, nur das iv.
FROM annoiv.gesamt_t t
where gid in ('H','R') 
```

In [2]:
import csv  

Open and read the export file (csv). 

In [3]:
input_file = open('alex20210323/alex_20210323.csv', encoding='utf-8')
input_csv_file = csv.DictReader(input_file, delimiter='\t')   
data = []
for row in input_csv_file: 
   data.append(row)
input_file.close()

In case of a KeyError please check the header entries (dict keys) and correct if necessary (e.g. 'Key' or '\ufeff "Key"')

In [45]:
display(row) 

OrderedDict([('id', '301232'),
             ('aid', 'lga'),
             ('jahr', '1849'),
             ('ausdatvf', 'NULL'),
             ('ausdatsf', 'NULL'),
             ('gesdatvf', '31. Grud. 1849'),
             ('gesdatsf', '18491231'),
             ('wirdatvf', 'NULL'),
             ('wirdatsf', 'NULL'),
             ('gestyp', 'Obwieszczenie'),
             ('inhalt',
              'Obwieszczenie Rządu krajowego, że postanowienia o wymiarze i poborze opłaty konsumcyjnej od wyrobów cukrowych z płodów krajowych w krajach koronnych Galicyi, Bukowiny i Krakowa, z dniem 1. Lutego 1850 wchodzą w wykonanie'),
             ('stueck', 'V'),
             ('stuecknr', '5'),
             ('gesnr', '111'),
             ('seite', '104'),
             ('geschzahl', 'NULL'),
             ('abtnr', 'NULL'),
             ('inst', 'NULL'),
             ('ordner', 'lga/1849/18490604'),
             ('sprache', 'pol'),
             ('schrift', 'L'),
             ('gid', 'R'),
             ('bezei

Define functions to write attributes (add new if necessary).

In [ ]:
def title(row):
        if row['inhalt'] not in (None, ""):
            return "\ttitle = {" + row['inhalt'] + "},\n"
        else:
            return ""

In [ ]:
def year(row):
        if row['ausdatsf'] not in (None, ""):
            return "\tdate = {" + row['ausdatsf'][0:4] + "-" + row['ausdatsf'][4:6] + "-" + row['ausdatsf'][6:8] + "},\n"
        else:
            return ""

In [ ]:
def pages(row):
        if row['seite'] not in (None, ""):
            return "\tpages = {" + row['seite'] + "},\n"
        else:
            return ""

In [ ]:
def number(row):
        if row['gesnr'] not in (None, ""):
            return "\tnumber = {" + row['gesnr'] + "/" + row['ausdatsf'][0:4] + "},\n"
        else:
            return "" 

In [ ]:
def volume(row):
        if row['stuecknr'] not in (None, ""):
            return "\tvolume = {" + row['stuecknr'] + "},\n"
        else:
            return "" 

In [ ]:
def url(row):
        if row['bool_exists'] not in (None, "", "0"):
            return "\turl = {https://alex.onb.ac.at/cgi-content/alex?aid=" + row['aid'] + "&datum=" + row['ordner'][9:] + "&seite=" + row['seite'].zfill(8) + "},\n"
        else:
            return ""

In [ ]:
def abstract(row):
        return "\tabstract = {Ausgabedatum: " + row['ausdatsf'] + "\nGesetzesdatum: " + row['gesdatsf'] + "\nWirkungsdatum: " + row['wirdatsf'] + "},\n"        

In [ ]:
def shorttitle(row):
        if row['kurzform'] not in (None, ""):
            return "\tshorttitle = {" + row['kurzform'] + " Nr. " + row['gesnr'] + "/" + row['ausdatsf'][0:4] + "},\n"
        else:
            return ""

In [ ]:
def keywords(row):
    return "\tkeywords = {" + "LGBl., ALEX" + "},\n"    

In [ ]:
def language(row):
        if row['sprache'] not in (None, ""):
            return "\tlanguage = {" + row['sprache'] + "},\n"
        else:
            return ""

In [ ]:
def bibkey(row):
    return "@legislation{alexlgbl" + row['id'] + ",\n"

And now for some Zotero RDF

In [57]:
def title(row):
        if row['inhalt'] not in (None, ""):
            return "<dc:title>" + row['inhalt'] + "</dc:title>\n"
        else:
            return ""

In [5]:
def year(row):
        if row['ausdatsf'] not in (None, ""):
            return "<dc:date>" + row['ausdatsf'][0:4] + "-" + row['ausdatsf'][4:6] + "-" + row['ausdatsf'][6:8] + "</dc:date>\n"
        else:
            return ""

In [6]:
def pages(row):
        if row['seite'] not in (None, ""):
            return "<bib:pages>" + row['seite'] + "</bib:pages>\n"
        else:
            return ""

In [7]:
def number(row):
        if row['gesnr'] not in (None, ""):
            return "<prism:number>" + row['gesnr'] + "/" + row['ausdatsf'][0:4] + "</prism:number>\n"
        else:
            return "" 

In [8]:
def volume(row):
        return "" 

In [9]:
def url(row):
        if row['bool_exists'] not in (None, "", "0"):
            return "<dc:identifier><dcterms:URI><rdf:value>https://alex.onb.ac.at/cgi-content/alex?aid=" + row['aid'] + "&datum=" + row['ordner'][9:] + "&seite=" + row['seite'].zfill(8) + "</rdf:value></dcterms:URI></dc:identifier>\n"
        else:
            return ""

In [43]:
def abstract(row):
        return "<dcterms:abstract>" + str(row['stuecknr']) + ". Stück\nAusgabedatum: " + str(row['ausdatsf']) + "\nGesetzesdatum: " + str(row['gesdatsf']) + "\nWirkungsdatum: " + str(row['wirdatsf']) + "</dcterms:abstract>\n"

In [11]:
def shorttitle(row):
        if row['kurzform'] not in (None, ""):
            return "<dc:description>" + row['kurzform'] + " Nr. " + row['gesnr'] + "/" + row['ausdatsf'][0:4] + "</dc:description>\n"
        else:
            return ""

In [24]:
def keywords(row):
    return "<dc:subject>" + "Gesetze LGBl., ALEX" + "</dc:subject>\n"    

In [13]:
def language(row):
        if row['sprache'] not in (None, ""):
            return "<z:language>" + row['sprache'] + "</z:language>\n"
        else:
            return ""

In [39]:
def bibkey(row):
    if row['bool_exists'] not in (None, "", "0"):
        return '<bib:Legislation rdf:about="https://alex.onb.ac.at/cgi-content/alex?aid=' + row['aid'] + "&datum=" + row['ordner'][9:] + "&seite=" + row['seite'].zfill(8) + '">\n'
    else:
        return "<bib:Legislation rdf:about='" + row['id'] + "'>\n"

A function that calls all attributes. (Just for a better overview.)  
For author and editor, semicolons are exchanged with an "and".

In [27]:
def attributes(row):
    bibkey(row)
    title(row)
    year(row)
    pages(row)
    number(row)
    url(row)
    abstract(row)
    volume(row)#Stücknummer
    shorttitle(row)
    keywords(row)
    language(row)
    return bibkey(row) + title(row) + year(row) + number(row) + pages(row) + volume(row) + keywords(row) + url(row) + abstract(row) + shorttitle(row) + language(row) + "</bib:Legislation>\n"
    

A function that writes individual bibliography entries including all attributes. 

In [28]:
def convert_row(row):
    return attributes(row)

In [62]:
bib = '\n'.join([convert_row(row) for row in data[1:]])

In [47]:
print(bib[:20000])

<bib:Legislation rdf:about="https://alex.onb.ac.at/cgi-content/alex?aid=lgm&datum=18500004&seite=00000007">
<dc:title>Erlaß des k. k. innerösterr. küstenländ. Appellationsgerichtes, betreffend die rasche und gewissenhafte Handhabung der Strafgesetze und der kaiserlichen Verordnung vom 27. October 1849 über das Verfahren in Besitzstörungs-Streitigkeiten}</dc:title>
<dc:date>1850-01-22</dc:date>
<prism:number>2/1850</prism:number>
<bib:pages>7</bib:pages>
<dc:subject>Gesetze LGBl., ALEX</dc:subject>
<dc:identifier><dcterms:URI><rdf:value>https://alex.onb.ac.at/cgi-content/alex?aid=lgm&datum=18500004&seite=00000007</rdf:value></dcterms:URI></dc:identifier>
<dcterms:abstract>1. Stück
Ausgabedatum: 18500122
Gesetzesdatum: 18491231
Wirkungsdatum: 18500122</dcterms:abstract>
<dc:description>LGBl. Steiermark Nr. 2/1850</dc:description>
<z:language>deu</z:language>
</bib:Legislation>

<bib:Legislation rdf:about="https://alex.onb.ac.at/cgi-content/alex?aid=lgm&datum=18500004&seite=00000005">
<dc

Replace Item Type with BIB references.

In [ ]:
bib_rep_char = bib.replace("&", "\&").replace("#", "\#").replace("$", "\$").replace("%", "\%").replace("_", "\_").replace("~", "\~").replace("^", "\^")

In [63]:
rdf_escape = bib.replace("&", "&amp;")

In [ ]:
bib_final = ('% Encoding: UTF-8\n\n' + bib_rep_char + '\n\n@Comment{jabref-meta: databaseType:bibtex;}')

In [70]:
rdf_comment = '''<!-- 
Data Source: Austrian National Library/Digitale Services/ALEX
Database Extraction: Andreas Wieser/Iris Klawatsch
Conversion: Stephan Kurz, Austrian Academy of Sciences
Licence: CC-0 (This is metadata only, freely available -- and it's describing non-copyrightable legal material)
Thanks for letting us reuse this data! 
-->
'''

In [71]:
rdf_final = rdf_comment + '''<rdf:RDF
 xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
 xmlns:z="http://www.zotero.org/namespaces/export#"
 xmlns:dc="http://purl.org/dc/elements/1.1/"
 xmlns:dcterms="http://purl.org/dc/terms/"
 xmlns:bib="http://purl.org/net/biblio#"
 xmlns:prism="http://prismstandard.org/namespaces/1.2/basic/">
 ''' + rdf_escape + '</rdf:RDF>'

In [60]:
print(rdf_final[:10000])


<!-- 
Data: Austrian National Library/ALEX
Conversion: Stephan Kurz, Austrian Academy of Sciences
Thanks for letting us reuse this data! 
-->
<rdf:RDF
 xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
 xmlns:z="http://www.zotero.org/namespaces/export#"
 xmlns:dc="http://purl.org/dc/elements/1.1/"
 xmlns:dcterms="http://purl.org/dc/terms/"
 xmlns:bib="http://purl.org/net/biblio#"
 xmlns:prism="http://prismstandard.org/namespaces/1.2/basic/">
 <bib:Legislation rdf:about="https://alex.onb.ac.at/cgi-content/alex?aid=lgm&amp;datum=18500004&amp;seite=00000007">
<dc:title>Erlaß des k. k. innerösterr. küstenländ. Appellationsgerichtes, betreffend die rasche und gewissenhafte Handhabung der Strafgesetze und der kaiserlichen Verordnung vom 27. October 1849 über das Verfahren in Besitzstörungs-Streitigkeiten}</dc:title>
<dc:date>1850-01-22</dc:date>
<prism:number>2/1850</prism:number>
<bib:pages>7</bib:pages>
<dc:subject>Gesetze LGBl., ALEX</dc:subject>
<dc:identifier><dcterms:URI><rdf:va

Write a bibtex bibliography file.

In [ ]:
f = open("lgbl_all.bib", "w", encoding="utf8")
f.write(bib_final)
f.close()

Write a Zotero RDF bibliography file.

In [72]:
f = open("lgbl_all.rdf", "w", encoding="utf8")
f.write(rdf_final)
f.close()